In this notebook, the autoencoder architecture is used to develop an anomaly detection algorithm. 
   
The training data are generated by computing a spectrogram from each signal and extracting features from these.

    Network Architecture is described below:

    Input shape: 640
    Architecture:
        Dense layer #1
            Dense layer (units: 64)
            Activation (ReLU)
        Dense layer #2
            Dense layer (units: 64)
            Activation (ReLU)
        Bottleneck layer
            Dense layer (units: 8)
            Batch Normalization
            Activation (ReLU)
        Dense layer #5
            Dense layer (units: 64)
            Activation (ReLU)
        Dense layer #6
            Dense layer (units: 64)
            Activation (ReLU)
        Output layer
            Dense layer (units: 640)
            
    Learning (epochs: 200, batch size: 256, data shuffling between epochs)
        Optimizer: Adam (learning rate: 0.001)
        
        
        
Model results shared below:

Accuracy: 95.83%, Precision: 91.67%, Recall: 100.00%, F1: 95.65%



In [1]:
import glob
import os
import warnings
import numpy as np
from bokeh.io import export_svgs, output_notebook
from bokeh.models import BoxAnnotation, ColumnDataSource, HoverTool
from bokeh.plotting import figure, show

from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
    average_precision_score,
    precision_recall_curve,
    roc_auc_score,
    roc_curve,
)
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

from utils import sound_tools
from utils.misc import build_files_list, dump_pickle, load_pickle
from utils.sound_utils import extract_signal_features, generate_dataset, load_sound_file
from utils.measuring_performance import (
    get_prediction,
    plot_confusion_matrix,
    plot_histogram_by_class,
    plot_loss_per_epoch,
    plot_pr_curve,
    plot_roc_curve,
)

/home/inovako/.local/lib/python3.6/site-packages/numba/core/errors.py:154: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)


In [2]:
import tensorflow as tf
from tensorflow.keras import Input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.python.client import device_lib

In [3]:
DATA_PATH = "./dataset"
IMAGE_PATH = "./img"
MODEL_PATH = "./models"

os.makedirs(os.path.join(DATA_PATH, "dataset"), exist_ok=True)

# Data Splitting and Preprocessing

Only normal sound signals are used for training set. 

In [4]:
normal_files, abnormal_files = build_files_list(root_dir=DATA_PATH)
normal_labels = np.zeros(len(normal_files))
abnormal_labels = np.ones(len(abnormal_files))

train_files, test_files, train_labels, test_labels = train_test_split(
    normal_files, normal_labels, train_size=0.8, random_state=42, shuffle=True
)

test_files = np.concatenate((test_files, abnormal_files), axis=0)
test_labels = np.concatenate((test_labels, abnormal_labels), axis=0)

test_indices = np.arange(len(test_files))
np.random.shuffle(test_indices)

test_files = test_files[test_indices]
test_labels = test_labels[test_indices]

print(
    f"Train set has {train_labels.shape[0]} signals including abnormal {train_labels.sum():.0f} signals, \
but test set has {test_labels.shape[0]} signals including abnormal {test_labels.sum():.0f} signals."
)

Train set has 48 signals including abnormal 0 signals, but test set has 24 signals including abnormal 11 signals.


In [5]:
dataset = {
    "train_files": train_files,
    "test_files": test_files,
    "train_labels": train_labels,
    "test_labels": test_labels,
}

In [6]:
for key, values in dataset.items():
    file_name = os.path.join(DATA_PATH, "dataset", key +  ".txt")
    with open(file_name, "w") as f:
        for item in values:
            f.write(str(item) + "\n")

Acoustic feature parameter

In [7]:
n_fft = 2048
hop_length = 512
n_mels = 64
frames = 10

In [8]:
train_data_path = os.path.join(DATA_PATH, "dataset", "train_data" + ".pkl")

if os.path.exists(train_data_path):
    print("Train data already exists, loading from file...")
    train_data = load_pickle(train_data_path)

else:
    train_data = generate_dataset(
        train_files, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels, frames=frames
    )
    print("Saving train data to disk...")
    dump_pickle(train_data_path, train_data)
    print("Done.")

print(f"Train data has a {train_data.shape} shape.")

Train data already exists, loading from file...
Train data has a (269616, 640) shape.


# Model Training and Prediction with AutoEncoder

In [9]:
def autoencoder(input_dims, model_name=None):
    input_layer = Input(shape=(input_dims,))
    output = Dense(64, activation="relu")(input_layer)
    output = Dense(64, activation="relu")(output)
    output = Dense(8, activation="relu")(output)
    output = Dense(64, activation="relu")(output)
    output = Dense(64, activation="relu")(output)
    output = Dense(input_dims, activation=None)(output)

    return Model(inputs=input_layer, outputs=output, name=model_name)

In [10]:
MODEL_NAME = "AutoEncoder"
MODEL_PATH = "/home/inovako/Documents/audio/models/AutoEncoder.h5"
model = autoencoder(n_mels * frames, model_name=MODEL_NAME)
print(model.summary())

Model: "AutoEncoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 640)]             0         
_________________________________________________________________
dense (Dense)                (None, 64)                41024     
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 520       
_________________________________________________________________
dense_3 (Dense)              (None, 64)                576       
_________________________________________________________________
dense_4 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_5 (Dense)              (None, 640)               

In [11]:
%%time
batch_size = 256
epochs = 200

model.compile(
    optimizer=Adam(learning_rate=1e-03),
    loss="mean_squared_error"
)

history = model.fit(
    train_data,
    train_data,
    batch_size=batch_size,
    epochs=epochs,
    verbose=False,
    callbacks=[EarlyStopping(monitor="val_loss", patience=10)],
    validation_split=0.1,
    shuffle=True
)

model.save(MODEL_PATH)

CPU times: user 4min 15s, sys: 20.1 s, total: 4min 35s
Wall time: 1min 11s


In [12]:
plot_loss_per_epoch(
    history, model_name=MODEL_NAME, file_name=os.path.join(IMAGE_PATH, "model_loss.svg")
)

![Loss plot](./img/model_loss.svg)

In [13]:
recon_errors = []

for index in tqdm(range(len(test_files))):
    signal, sr = load_sound_file(test_files[index])

    features = extract_signal_features(
        signal, sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels, frames=frames
    )

    predictions = model.predict(features)
    mse = np.mean(np.mean(np.square(features - predictions), axis=1))
    recon_errors.append(mse)


  0%|          | 0/24 [00:00<?, ?it/s]

# Model Evaluation


In [15]:
stack = np.column_stack((range(len(recon_errors)), recon_errors))
score_false = stack[test_labels == 0][:, 1]
score_true = stack[test_labels == 1][:, 1]

plot_histogram_by_class(
    score_false,
    score_true,
    bins=[20, 30],
    model_name=MODEL_NAME,
    file_name=os.path.join(IMAGE_PATH, "recon_error_dist.svg"),
)

![Reconstruction Error](./img/recon_error_dist.svg)

In [20]:
THRESHOLD_MIN = 6.0
THRESHOLD_MAX = 10.0
THRESHOLD_STEP = 0.5

p = figure(
    plot_width=600,
    plot_height=400,
    title=f"{MODEL_NAME}: Threshold Range Exploration",
    x_axis_label="Samples",
    y_axis_label="Reconstruction Error",
)

source = ColumnDataSource(
    dict(index=stack[test_labels == 0][:, 0], error=stack[test_labels == 0][:, 1])
)
p.scatter(
    "index",
    "error",
    fill_alpha=0.6,
    fill_color="crimson",
    line_color=None,
    legend_label="Normal Signals",
    source=source,
)

source = ColumnDataSource(
    dict(index=stack[test_labels == 1][:, 0], error=stack[test_labels == 1][:, 1])
)
p.scatter(
    "index",
    "error",
    fill_alpha=0.6,
    fill_color="indigo",
    line_color=None,
    legend_label="Abnormal Signals",
    source=source,
)

source = ColumnDataSource(
    data=dict(
        index=stack[:, 0],
        threshold_min=np.repeat(THRESHOLD_MIN, stack.shape[0]),
        threshold_max=np.repeat(THRESHOLD_MAX, stack.shape[0]),
    )
)

box = BoxAnnotation(
    bottom=THRESHOLD_MIN,
    top=THRESHOLD_MAX,
    fill_alpha=0.1,
    fill_color="magenta",
    line_color="darkmagenta",
    line_width=1.0,
)
p.add_layout(box)

p.legend.label_text_font_size = "8pt"
p.legend.location = "top_right"
p.title.align = "center"
p.title.text_font_size = "12pt"

p.add_tools(HoverTool(tooltips=[("index", "@index"), ("error", "@error")]))
show(p)

p.output_backend = "svg"
_ = export_svgs(p, filename=os.path.join(IMAGE_PATH, "thr_range_exp.svg"))

![Threshold range exploration](./img/thr_range_exp.svg)

In [21]:

thresholds = np.arange(THRESHOLD_MIN, THRESHOLD_MAX + THRESHOLD_STEP, THRESHOLD_STEP)
errors = []

for threshold in thresholds:
    predictions = get_prediction(stack[:, 1], threshold=threshold)
    conf_mat = confusion_matrix(test_labels, predictions)
    errors.append([threshold, conf_mat[1, 0], conf_mat[0, 1]])

errors = np.array(errors)



In [22]:
THRESHOLD = 6
predictions = get_prediction(stack[:, 1], threshold=THRESHOLD)

plot_confusion_matrix(
    confusion_matrix(test_labels, predictions),
    model_name=MODEL_NAME,
    file_name=os.path.join(IMAGE_PATH, "conf_mat.svg"),
)

![Confusion Matrix](./img/conf_mat.svg)

In [23]:
print(
    f"Accuracy: {accuracy_score(test_labels, predictions):.2%}, \
Precision: {precision_score(test_labels, predictions):.2%}, \
Recall: {recall_score(test_labels, predictions):.2%}, \
F1: {f1_score(test_labels, predictions):.2%}"
)

Accuracy: 91.67%, Precision: 90.91%, Recall: 90.91%, F1: 90.91%
